## Prepare the data

### Setup the tokenizer

In [1]:
from transformers import AutoTokenizer

MPATH = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(MPATH)

### Setup the data modules

In [2]:
from datautils import PersuationSchemeDataModule, PersuationSchemeBatcher


DPATH = r"..\data\persuasionforgood_corpus"

batcher = PersuationSchemeBatcher(tokenizer)
dm = PersuationSchemeDataModule(
    DPATH,
    batcher=batcher,
    batch_size=64
)

# To harvest the id2label dict
dm.setup('test')
id2label = dm.test_dataset.id2label

Building the Utterance Labels: 100%|██████████| 1534/1534 [00:00<00:00, 66691.50it/s]


# Prepare the model

In [3]:
from modelling import PersuasionSchemeClassifier


SAVE_PATH = f"../models/persuasion-schemes-classifier"
CKPT = ""

model = PersuasionSchemeClassifier(
    MPATH,
    id2label=id2label
)
print("Loaded fresh model")

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

Loaded fresh model


In [4]:
tokenizer.save_pretrained(SAVE_PATH)

('../models/persuasion-schemes-classifier\\tokenizer_config.json',
 '../models/persuasion-schemes-classifier\\special_tokens_map.json',
 '../models/persuasion-schemes-classifier\\vocab.json',
 '../models/persuasion-schemes-classifier\\merges.txt',
 '../models/persuasion-schemes-classifier\\added_tokens.json',
 '../models/persuasion-schemes-classifier\\tokenizer.json')

### Setup Callbacks

In [5]:
from pytorch_lightning.callbacks import(
    EarlyStopping,
    ModelCheckpoint,
    RichModelSummary
)

checkpoint_callback = ModelCheckpoint(
    SAVE_PATH,
    filename=f'epoch-{{epoch}}-{{val_loss:.2f}}',
    monitor='val_overall_f1_score',
    save_weights_only=True,
    mode='max'
)
early_stop_callback = EarlyStopping(
    monitor="val_overall_f1_score",
    min_delta=1e-4, patience=8,
    verbose=False,
    mode="max"
)

# Train the model

In [6]:
from pytorch_lightning import Trainer, seed_everything


seed_everything(42, workers=True)

trainer = Trainer(
    max_epochs=-1,
    deterministic=True,
    accumulate_grad_batches=2,
    callbacks=[checkpoint_callback, early_stop_callback, RichModelSummary()],
    accelerator='gpu',
    log_every_n_steps=16
)

trainer.fit(model, dm)

Global seed set to 42
Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\envs\expts\Lib\site-packages\pytorch_lightning\trainer\connectors\logger_connector\logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
Building the Utterance Labels: 100%|██████████| 1151/1151 [00:00<00:00, 67696.55it/s]
d:\

┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name           ┃ Type                             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ seq_classifier │ RobertaForSequenceClassification │  124 M │
└───┴────────────────┴──────────────────────────────────┴────────┘

Trainable params: 124 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 124 M                                                                                                
Total estimated model params size (MB): 498

Sanity Checking: 0it [00:00, ?it/s]

d:\envs\expts\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
d:\envs\expts\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

### Evaluate model

In [11]:
model = PersuasionSchemeClassifier.load_from_checkpoint(r'D:\repos\cse546-rl-assignments\grp_prj\models\persuasion-schemes-classifier\epoch-epoch=18-val_loss=0.56.ckpt')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [12]:
import torch
with torch.no_grad():
    trainer.test(model, dm)

Building the Utterance Labels: 100%|██████████| 1534/1534 [00:00<00:00, 61369.72it/s]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃              Runningstage.testing metric               ┃                      DataLoader 0                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_granular_f1_score_acknowledgement         │                   0.4965677857398987                   │
│         test_granular_f1_score_agree-donation          │                   0.4566958248615265                   │
│         test_granular_f1_score_ask-donate-more         │                   0.1794871836900711                   │
│       test_granular_f1_score_ask-donation-amount       │                   0.5679518580436707                   │
│     test_granular_f1_score_ask-donation-procedure      │                  0.43523934483528137                   │
│      test_granular_f1_score_ask-not-donate-reason      │                          0.0                           │
│          test_granular_f1_score_ask-org-info           │                   0.6838791370391846                   │
│ test_granular_f1_score_ask-persuader-donation-intenti… │                  0.23502826690673828                   │
│             test_granular_f1_score_closing             │                   0.5285803079605103                   │
│         test_granular_f1_score_comment-partner         │                  0.027813995257019997                  │
│        test_granular_f1_score_confirm-donation         │                  0.27833864092826843                   │
│       test_granular_f1_score_credibility-appeal        │                    0.72699373960495                    │
│        test_granular_f1_score_disagree-donation        │                   0.2967405319213867                   │
│     test_granular_f1_score_disagree-donation-more      │                          0.0                           │
│      test_granular_f1_score_donation-information       │                   0.6261316537857056                   │
│         test_granular_f1_score_emotion-appeal          │                   0.5566400289535522                   │
│        test_granular_f1_score_foot-in-the-door         │                   0.4109848737716675                   │
│            test_granular_f1_score_greeting             │                   0.901367723941803                    │
│         test_granular_f1_score_logical-appeal          │                   0.4560949504375458                   │
│  test_granular_f1_score_negative-reaction-to-donation  │                   0.3249022364616394                   │
│       test_granular_f1_score_negative-to-inquiry       │                   0.4736429452896118                   │
│  test_granular_f1_score_neutral-reaction-to-donation   │                  0.027813995257019997                  │
│       test_granular_f1_score_neutral-to-inquiry        │                   0.1126466765999794                   │
│            test_granular_f1_score_off-task             │                   0.1296330839395523                   │
│              test_granular_f1_score_other              │                   0.3900463879108429                   │
│    test_granular_f1_score_personal-related-inquiry     │                   0.5120800137519836                   │
│         test_granular_f1_score_personal-story          │                  0.16410258412361145                   │
│  test_granular_f1_score_positive-reaction-to-donation  │                  0.47991541028022766                   │
│       test_granular_f1_score_positive-to-inquiry       │                   0.4047980010509491                   │
│           test_granular_f1_score_praise-user           │                  0.35108548402786255                   │
│     test_granular_f1_score_proposition-of-donation     │                   0.5613510608673096                   │
│     test_granular_f1_score_provide-donation-amount    

### Push to hub

In [13]:
# model.seq_classifier.push_to_hub('nasheed/rl-grp-prj-per-cls')

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/nasheed/rl-grp-prj-per-cls/commit/83617b5381eef42d71b06a2706f1ea3e900f3f19', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='83617b5381eef42d71b06a2706f1ea3e900f3f19', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# tokenizer.push_to_hub('nasheed/rl-grp-prj-per-cls')